In [8]:
import subprocess
from pydub import AudioSegment
import math
import os

def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-y",
        "-i",
        video_path,
        "-vn",
        "-acodec",
        "libmp3lame",
        "-b:a",
        "128k",
        audio_path,
    ]
    subprocess.run(command, check=True)

def chunk_audio(audio_path, chunk_duration, output_dir):
    # 오디오 파일 로드
    audio = AudioSegment.from_mp3(audio_path)

    # 청크 길이 계산 (밀리초 단위)
    chunk_length_ms = chunk_duration * 60 * 1000
    
    # 청크 수 계산
    chunks = math.ceil(len(audio) / chunk_length_ms)

    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # 오디오 청크 생성 및 저장
    for i in range(chunks):
        start_time = i * chunk_length_ms
        end_time = min((i + 1) * chunk_length_ms, len(audio))
        
        chunk = audio[start_time:end_time]
        chunk.export(f"{output_dir}/chunk_{i}.mp3", format="mp3")

    return chunks

# 메인 프로세스
video_path = "./files/podcast.mp4"
audio_path = "./files/extracted_audio.mp3"
chunk_output_dir = "./files/chunks"
chunk_duration = 10  # 분 단위

# 비디오에서 오디오 추출
print("비디오에서 오디오 추출 중...")
extract_audio_from_video(video_path, audio_path)
print("오디오 추출 완료")

# 오디오를 청크로 분할
print("오디오를 청크로 분할 중...")
num_chunks = chunk_audio(audio_path, chunk_duration, chunk_output_dir)
print(f"오디오 분할 완료. 총 {num_chunks}개의 청크가 생성되었습니다.")